# Digit Classification using Dense Neural Network (DNN) 

## Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Upload and Explore Dataset
[MNIST](http://yann.lecun.com/exdb/mnist/) handwritten digits dataset

The MNIST database of handwritten digits, available from this [page](http://yann.lecun.com/exdb/mnist/), has a training set of 60,000 28x28 grayscale images of the 10 digits along a test set of 10,000 images. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.


In [ ]:
data = tf.keras.datasets.mnist

(training_images, training_labels), (val_images, val_labels) = data.load_data()

In [ ]:
print(training_images.shape)
print(training_labels.shape)

In [ ]:
print(val_images.shape)
print(val_labels.shape)

### Exploring Labels

In [ ]:
training_labels

It is possible to keep training labels as "numbers", but in this case when compiling the model, you should use: `loss="sparse_categorical_crossentropy".`

**And how about changing labels to categorical?**

In [ ]:
training_labels[:10]

In [ ]:
from tensorflow.keras.utils import to_categorical
to_categorical(training_labels[:10])

When labels are defined as categories, when compiling the model you should use: `loss="categorical_crossentropy".`

### Exploring images

In [ ]:
np.set_printoptions(linewidth=200)
print(training_images[2])

In [9]:
training_labels[2]

4

In [ ]:
img = 2
print("     Label of image {} is: {}".format(img, training_labels[img]))
plt.imshow(training_images[img]);

In [ ]:
training_images.max()

In [ ]:
training_images.min()

### Preprocessing Data 

**Normalizing Data**: 
We notice that all of the values in the number are between 0 and 255. If we are training a neural network, for various reasons it's easier if we treat all values as between 0 and 1, a process called 'normalizing'.

In [13]:
training_images  = training_images / 255.0
val_images = val_images / 255.0

In [ ]:
print(training_images[2])

In [ ]:
training_images.max()

In [ ]:
plt.imshow(training_images[2]);

## Define and Compile Model

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    tf.keras.layers.Dense(20, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])
model.summary()

In [18]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # Labels are not as an array 
    metrics=['accuracy'] # Calculates how often predictions equal labels
    )

## Train the model

In [ ]:
%%time
history = model.fit(
    training_images, 
    training_labels, 
    epochs=20, 
    )

Inspecting the model

In [ ]:
train_eval = model.evaluate(training_images, training_labels)
print ("Training data Accuracy: {:.1f}%".format(train_eval[1]*100))

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuraccy'], loc='upper left')
plt.show()

## Testing the trained model

Using `model.evaluate`, you can get metrics for a test set. In this case we only have a training set and a validation set, so we can try it out with the validation set. The accuracy will be slightly lower, at maybe 96%. This is because the model hasn't previously seen this data and may not be fully generalized for all data. Still it's a pretty good score.
You can also predict images, and compare against their actual label. The [0] image in the set is a number 7, and here you can see that neuron 7 has a 9.9e-1 (99%+) probability, so it got it right!

In [ ]:
test_eval = model.evaluate(val_images, val_labels)
print ("Testing data Accuracy: {:.1f}%".format(test_eval[1]*100))

In [ ]:
plt.imshow(val_images[0]);

In [ ]:
print(val_labels[0])

In [ ]:
classifications = model.predict(val_images)
print(classifications[0])

In [ ]:
# Returns the indices of the maximum values along an axis.
np.argmax(classifications[0])

## Inspecting Errors

In [ ]:
predict_errors = []
for i in range(len(val_labels)):
  predicted_label = np.argmax(classifications[i])
  if predicted_label != val_labels[i]:
    predict_errors.append(i)

print(1-(len(predict_errors)/len(val_labels)))

In [ ]:
predict_errors[:10]

In [ ]:
i = predict_errors[5]
predicted_label = np.argmax(classifications[i])
print("    Real label of image {} is: {}".format(i, val_labels[i]))
print(" Predict label of image {} is: {}".format(i, predicted_label))
plt.imshow(val_images[i]);

## Modify to inspect learned values

This code is identical, except that the layers are named prior to adding to the sequential. This allows us to inspect their learned parameters later.

In [ ]:
layer_1 = tf.keras.layers.Dense(20, activation=tf.nn.relu)
layer_2 = tf.keras.layers.Dense(10, activation=tf.nn.softmax)

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    layer_1,
                                    layer_2])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, 
          training_labels, 
          epochs=20,
          verbose=0
          )

test_eval = model.evaluate(val_images, val_labels)
print ("Testing data Accuracy: {:.1f}%".format(test_eval[1]*100))

In [ ]:
classifications = model.predict(val_images)
print(classifications[0])
print(np.argmax(classifications[0]))
print(val_labels[0])

Inspect Layer 1

In [ ]:
print("Layer 1 Ws ==>", layer_1.get_weights()[0].size)
print("Layer 1 bs ==>", layer_1.get_weights()[1].size)

The `get_weights()[0]` contains the weights (Ws). Why we have 15,680 w's on layer 1? Recall that there are 20 neurons in the first layer and, that the images are 28x28 pixels, which is 784. So, if you multiply 784 x 20 you get 15,680.

This layer has 20 neurons, and each neuron learns a W parameter for each pixel. So instead of y=Mx+c, we have 
y=M1X1+M2X2+M3X3+....+M784X784+C in every neuron! Every pixel has a weight in every neuron. Those weights are multiplied by the pixel value, summed up, and given a bias.

The `get_weights()[1]` contains the biases (bs), one for each of the 20 neurons in this layer.


Inspect Layer 2

In [ ]:
print("Layer 2 Ws ==>", layer_2.get_weights()[0].size)
print("Layer 2 bs ==>", layer_2.get_weights()[1].size)

Layer 2 has 200 Ws. This is because are 10 neurons in this layer, but there are 20 neurons in the previous layer. So, each neuron in this layer will learn a weight for the incoming value from the previous layer. So, for example, the if the first neuron in this layer is N21, and the neurons output from the previous layers are N11-N120, then this neuron will have 20 weights (W1-W20) and it will calculate its output to be:

`W1N11+W2N12+W3N13+...+W20N120+Bias`

So each of these weights will be learned as will the bias, for every neuron.

Note that N11 refers to Layer 1 Neuron 1.

...and as expected there are 10 elements in the second array, representing the 10 biases for the 10 neurons.
Hopefully this helps you see how the element of a simple neuron containing y=mx+c can be expanded greatly into a deep neural network, and that DNN can learn the parameters that match the 784 pixels of an image to their output!